In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster, HeatMap
import branca
import base64

df = pd.read_csv(r"D:\MS\Indiana University Bloomington\Classes\ENGR-E 583 Information Visualization (Dr. Katy Börner & Michael Ginda)\Client Project\Week 5\final_preprocessed_dataset.csv")
df["Start Date"] = pd.to_datetime(df["Start Date"], errors="coerce")
df["End Date"] = pd.to_datetime(df["End Date"], errors="coerce")
df = df.dropna(subset=["Latitude", "Longitude"])
df = df.fillna("N/A")

def get_interval(year):
    if 2005 <= year <= 2009:
        return "2005–2009"
    elif 2010 <= year <= 2014:
        return "2010–2014"
    elif 2015 <= year <= 2019:
        return "2015–2019"
    elif 2020 <= year <= 2025:
        return "2020–2025"
    else:
        return "Other"

df["Year Interval"] = df["Year Started"].apply(get_interval)

map_center = [df["Latitude"].median(), df["Longitude"].median()]
m = folium.Map(location=map_center, zoom_start=2, tiles="cartodbpositron")
timeline = branca.element.Figure()
timeline.add_child(m)

main_cluster = MarkerCluster(name="All Exhibit Venues").add_to(m)
for _, row in df.iterrows():
    popup_text = f"""
    <b>Title:</b> {row['Title']}<br><br>
    <b>Venue:</b> {row['Venue']}<br><br>
    <b>Start Date:</b> {pd.to_datetime(row['Start Date']).strftime('%Y-%m-%d')}<br><br>
    <b>End Date:</b> {pd.to_datetime(row['End Date']).strftime('%Y-%m-%d')}<br><br>
    <b>Country:</b> {row['Country']}<br><br>
    <b>State:</b> {row['State']}<br><br>
    <b>City:</b> {row['City']}<br><br>
    <b>Organizer:</b> {row['Organizer']}<br><br>
    <b>Event Duration (Days):</b> {row['Event Duration (Days)']}
    """
    folium.Marker(
        location=(row["Latitude"], row["Longitude"]),
        popup=folium.Popup(popup_text, max_width=400),
        tooltip=row["Title"]
    ).add_to(main_cluster)

heat_data = df[['Latitude', 'Longitude']].dropna().values.tolist()
HeatMap(heat_data, name="Heatmap", show=False).add_to(m)

intervals = df["Year Interval"].unique()
for interval in sorted(intervals):
    if interval == "Other":
        continue
    interval_df = df[df["Year Interval"] == interval]
    fg = folium.FeatureGroup(name=f"{interval}", show=False).add_to(m)
    cluster = MarkerCluster().add_to(fg)
    for _, row in interval_df.iterrows():
        popup_text = f"""
        <b>Title:</b> {row['Title']}<br><br>
        <b>Venue:</b> {row['Venue']}<br><br>
        <b>Start Date:</b> {pd.to_datetime(row['Start Date']).strftime('%Y-%m-%d')}<br><br>
        <b>End Date:</b> {pd.to_datetime(row['End Date']).strftime('%Y-%m-%d')}<br><br>
        <b>Country:</b> {row['Country']}<br><br>
        <b>State:</b> {row['State']}<br><br>
        <b>City:</b> {row['City']}<br><br>
        <b>Organizer:</b> {row['Organizer']}<br><br>
        <b>Event Duration (Days):</b> {row['Event Duration (Days)']}
        """
        folium.Marker(
            location=(row["Latitude"], row["Longitude"]),
            popup=folium.Popup(popup_text, max_width=400),
            tooltip=row["Title"]
        ).add_to(cluster)

total_events = len(df)
total_html = f"""
<div style="position: fixed;
            bottom: 40px; left: 50px; width: 260px;
            background-color: white; border:2px solid grey;
            z-index:9999; font-size:16px; padding: 10px;">
    <b>Total Events (2005–2025):</b> {total_events}<br>
</div>
"""
m.get_root().html.add_child(folium.Element(total_html))

with open(r"D:\MS\Indiana University Bloomington\Classes\ENGR-E 583 Information Visualization (Dr. Katy Börner & Michael Ginda)\Client Project\CNS Logo.png", 'rb') as f:
    encoded_image = base64.b64encode(f.read()).decode('utf-8')

logo_html = f"""
<div style="position: fixed;
            bottom: 40px; right: 50px; width: 200px; height: 100px;
            background-color: white; border:2px solid grey;
            padding: 10px;
            background-image: url('data:image/png;base64,{encoded_image}');
            background-size: contain;
            background-repeat: no-repeat;
            background-position: center;
            z-index:9999;">
</div>
"""
m.get_root().html.add_child(folium.Element(logo_html))

folium.LayerControl(collapsed=False).add_to(m)

title_html = """
<div style="position: fixed;
            top: 20px; left: 50%; transform: translateX(-50%);
            background-color: white; border: 1.5px solid grey;
            z-index:9999; font-size:16px; padding: 10px 20px; text-align: center; width: 600px;">
    <b>Global Exhibit Venues (2005–2025)</b><br>
    <hr style="margin: 6px 0;">
    <span style="font-size:12px; color:#555;">
        Explore 20 years of exhibit venues worldwide — clustered by location and organized by time.
    </span>
</div>
"""
m.get_root().html.add_child(folium.Element(title_html))

js = """
<script>
setTimeout(function() {
    const overlays = document.querySelectorAll('.leaflet-control-layers-overlays input');
    overlays.forEach(input => {
        input.addEventListener('change', function() {
            if (this.checked) {
                overlays.forEach(other => {
                    if (other !== this) {
                        if (other.checked) other.click();
                    }
                });
            }
        });
    });
}, 1000);
</script>
"""
m.get_root().html.add_child(folium.Element(js))

m.save("Global Exhibit Venues.html")